In [1]:
import os
import copy
import torch
import detectron2
from detectron2.data import detection_utils as utils
from detectron2.utils.logger import setup_logger
setup_logger()

from detectron2 import model_zoo
from detectron2.config import get_cfg, CfgNode
from detectron2.engine import DefaultTrainer
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.evaluation import COCOEvaluator
from detectron2.data import build_detection_test_loader, build_detection_train_loader

In [2]:
# Register Dataset
try:
    register_coco_instances('coco_trash_train', {}, '/data/ephemeral/home/workspace/dataset/train.json', '/data/ephemeral/home/workspace/dataset/')
except AssertionError:
    pass

try:
    register_coco_instances('coco_trash_test', {}, '/data/ephemeral/home/workspace/dataset/test.json', '/data/ephemeral/home/workspace/dataset/')
except AssertionError:
    pass

MetadataCatalog.get('coco_trash_train').thing_classes = ["General trash", "Paper", "Paper pack", "Metal", 
                                                         "Glass", "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing"]

In [3]:
# config 불러오기
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file('COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml'))

In [4]:
# config 수정하기
cfg.DATASETS.TRAIN = ('coco_trash_train',)
cfg.DATASETS.TEST = ('coco_trash_test',)

cfg.DATALOADER.NUM_WOREKRS = 2

cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url('COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml')

cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.001
cfg.SOLVER.MAX_ITER = 15000
cfg.SOLVER.STEPS = (8000,12000)
cfg.SOLVER.GAMMA = 0.005
cfg.SOLVER.CHECKPOINT_PERIOD = 3000

cfg.OUTPUT_DIR = './output'

cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 10

cfg.TEST.EVAL_PERIOD = 3000

# mlflow config 추가 
cfg.MLFLOW = CfgNode()
cfg.MLFLOW.EXPERIMENT_NAME = "detectron2_faster_rcnn"
cfg.MLFLOW.RUN_NAME = "#0_detectron2_baseline_training"
cfg.MLFLOW.TRACKING_URI = "http://10.28.224.171:30280"


In [5]:
from detectron2.engine import HookBase
import mlflow

class MLflowHook(HookBase):
    """
    custom hook class를 사용하여 mlflow에 model artifacts와 metrics, parameters 추가 
    """
    def __init__(self, cfg) -> None:
        super().__init__()
        self.cfg = cfg.clone()
    
    # tracking server URI, experiment 이름, run 이름을 설정하고 실행을 시작
    def before_train(self):
        with torch.no_grad():
            mlflow.set_tracking_uri(self.cfg.MLFLOW.TRACKING_URI)
            mlflow.set_experiment(self.cfg.MLFLOW.EXPERIMENT_NAME)
            mlflow.start_run(run_name=self.cfg.MLFLOW.RUN_NAME)
        for k, v in self.cfg.items():
                mlflow.log_param(k, v) 
                
    # iteration step마다 Detectron2의 EventStorage에서 latest training metrics을 요청
    def after_step(self):
        with torch.no_grad():
            latest_metrics = self.trainer.storage.latest()
            for k, v in latest_metrics.items():
                mlflow.log_metric(key=k, value=v[0], step=v[1])
    
    # Detectron2 구성을 YAML 파일에 덤프하고 마지막으로 모든 출력 파일(config YAML 포함)을 MLflow에 로깅      
    def after_train(self):
        with torch.no_grad():
            with open(os.path.join(self.cfg.OUTPUT_DIR, "model-config.yaml"), "w") as f:
                f.write(self.cfg.dump())
            mlflow.log_artifacts(self.cfg.OUTPUT_DIR)

In [6]:
# mapper - input data를 어떤 형식으로 return할지 (따라서 augmnentation 등 데이터 전처리 포함 됨)
import detectron2.data.transforms as T

def MyMapper(dataset_dict):
    dataset_dict = copy.deepcopy(dataset_dict)
    image = utils.read_image(dataset_dict['file_name'], format='BGR')
    
    transform_list = [
        T.RandomFlip(prob=0.5, horizontal=False, vertical=True),
        T.RandomBrightness(0.8, 1.8),
        T.RandomContrast(0.6, 1.3)
    ]
    
    image, transforms = T.apply_transform_gens(transform_list, image)
    
    dataset_dict['image'] = torch.as_tensor(image.transpose(2,0,1).astype('float32'))
    
    annos = [
        utils.transform_instance_annotations(obj, transforms, image.shape[:2])
        for obj in dataset_dict.pop('annotations')
        if obj.get('iscrowd', 0) == 0
    ]
    
    instances = utils.annotations_to_instances(annos, image.shape[:2])
    dataset_dict['instances'] = utils.filter_empty_instances(instances)
    
    return dataset_dict

In [7]:
# trainer - DefaultTrainer를 상속
class MyTrainer(DefaultTrainer):
    
    @classmethod
    def build_train_loader(cls, cfg, sampler=None):
        return build_detection_train_loader(
        cfg, mapper = MyMapper, sampler = sampler
        )
    
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            os.makedirs('./output_eval', exist_ok = True)
            output_folder = './output_eval'
            
        return COCOEvaluator(dataset_name, cfg, False, output_folder)

In [8]:
# train
os.makedirs(cfg.OUTPUT_DIR, exist_ok = True)

mlflow_hook = MLflowHook(cfg)

trainer = MyTrainer(cfg)
trainer.register_hooks(hooks=[mlflow_hook])
trainer.resume_or_load(resume=False)
trainer.train()

[10/10 01:15:33 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (11, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (11,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (40, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (40,) in the model! You might want to double check if this is expected.
Some model parameters or buffers are not found in the checkpoint:
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias

[10/10 01:15:34 d2.engine.train_loop]: Starting training from iteration 0


2024/10/10 01:15:34 INFO mlflow.tracking.fluent: Experiment with name 'detectron2_faster_rcnn' does not exist. Creating a new experiment.
/opt/conda/lib/python3.10/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[10/10 01:15:47 d2.utils.events]:  eta: 2:01:10  iter: 19  total_loss: 3.186  loss_cls: 2.282  loss_box_reg: 0.7591  loss_rpn_cls: 0.09385  loss_rpn_loc: 0.02435    time: 0.4867  last_time: 0.4796  data_time: 0.0282  last_data_time: 0.0145   lr: 1.9981e-05  max_mem: 6696M
[10/10 01:15:58 d2.utils.events]:  eta: 2:00:11  iter: 39  total_loss: 2.807  loss_cls: 1.862  loss_box_reg: 0.7333  loss_rpn_cls: 0.05259  loss_rpn_loc: 0.02533    time: 0.4837  last_time: 0.4800  data_time: 0.0143  last_data_time: 0.0129   lr: 3.9961e-05  max_mem: 6696M
[10/10 01:16:09 d2.utils.events]:  eta: 2:00:01  iter: 59  total_loss: 1.973  loss_cls: 1.102  loss_box_reg: 0.6842  loss_rpn_cls: 0.05342  loss_rpn_loc: 0.0398    time: 0.4839  last_time: 0.4841  data_time: 0.0150  last_data_time: 0.0173   lr: 5.9941e-05  max_mem: 6696M
[10/10 01:16:20 d2.utils.events]:  eta: 1:59:57  iter: 79  total_loss: 1.695  loss_cls: 0.8372  loss_box_reg: 0.7189  loss_rpn_cls: 0.0351  loss_rpn_loc: 0.02802    time: 0.4836  las

KeyboardInterrupt: 